In [21]:
import pandas as pd
import requests
import requests_cache
import re
from bs4 import BeautifulSoup as bs
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
requests_cache.install_cache("cache4")

In [22]:
recipes = pd.read_json('full_format_recipes.json')
recipes.dropna(how='all', inplace=True)
# recipes.drop_duplicates('title', inplace=True) 


no_nutri = recipes[pd.isna(recipes.calories)]

In [2]:
no_nutri = no_nutri.sort_values('rating', ascending=False)
no_nutri.drop_duplicates('title', inplace=True)
no_nutri.sort_index(inplace=True)
no_nutri.to_json('no_nutri.json')


In [23]:
np.median(recipes.calories.dropna())
# np.nanmedian(pd.Series([0,1,2,10,np.nan]))

331.0

In [24]:
no_dup = recipes.drop_duplicates(subset=['title'])
no_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17775 entries, 0 to 20129
Data columns (total 11 columns):
calories       14088 non-null float64
categories     17775 non-null object
date           17775 non-null datetime64[ns]
desc           11917 non-null object
directions     17775 non-null object
fat            14044 non-null float64
ingredients    17775 non-null object
protein        14063 non-null float64
rating         17766 non-null float64
sodium         14087 non-null float64
title          17775 non-null object
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 1.6+ MB


In [72]:
no_dup[['calories', 'fat', 'sodium', 'protein']].describe()

,calories,fat,sodium,protein
count,1.408800e+04,1.404400e+04,1.408700e+04,14063.000000
mean,6.806129e+03,3.733730e+02,6.756175e+03,94.568513
std,3.802477e+05,2.166416e+04,3.534986e+05,3562.184004
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,2.080000e+02,9.000000e+00,9.200000e+01,3.000000
50%,3.510000e+02,1.800000e+01,3.100000e+02,9.000000
75%,6.050000e+02,3.500000e+01,7.400000e+02,29.000000
max,3.011122e+07,1.722763e+06,2.767511e+07,236489.000000


In [25]:
recipes.calories.median()

331.0

In [70]:
# recipes[recipes['title'] == 'Salted Water for Boiling ']
recipes.loc[285] == recipes.loc[10790] 
# type(recipes.loc[285, 'fat']) == type(recipes.loc[10790, 'fat'])
test = recipes.loc[recipes.duplicated(subset='title', keep=False)]
test.rating.describe()

count    4249.000000
mean        3.673070
std         1.459046
min         0.000000
25%         3.750000
50%         4.375000
75%         4.375000
max         5.000000
Name: rating, dtype: float64

In [71]:
test.sort_values('rating', ascending=False).drop_duplicates('title').sort_index().rating.describe()
# test.drop_duplicates('title')


count    1915.000000
mean        3.816906
std         1.351594
min         0.000000
25%         3.750000
50%         4.375000
75%         4.375000
max         5.000000
Name: rating, dtype: float64

In [24]:
no_dup[no_dup.title == 'Blackberry-Raspberry Sauce ']

,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title
118,129.0,"[Sauce, Blender, Dessert, Quick & Easy, Low So...",2004-08-20 04:00:00,Can be prepared in 45 minutes or less.,[In a small saucepan bring sugar and water to ...,0.0,"[1/2 cup sugar, 1/3 cup water, 1 cup picked-ov...",1.0,4.375,2.0,Blackberry-Raspberry Sauce


In [26]:
recipes[recipes['calories'] == 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 285 to 16115
Data columns (total 11 columns):
calories       13 non-null float64
categories     13 non-null object
date           13 non-null datetime64[ns]
desc           8 non-null object
directions     13 non-null object
fat            8 non-null float64
ingredients    13 non-null object
protein        8 non-null float64
rating         13 non-null float64
sodium         13 non-null float64
title          13 non-null object
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 1.2+ KB


In [40]:
no_dup[no_dup['calories'] < 10].loc[754,'date']

Timestamp('2005-10-21 20:16:44')

In [250]:
recipes[['calories', 'fat', 'sodium', 'protein']].describe()

,calories,fat,sodium,protein
count,1.597600e+04,1.590800e+04,1.597400e+04,15929.000000
mean,6.307857e+03,3.460975e+02,6.211474e+03,99.946199
std,3.585851e+05,2.043102e+04,3.328903e+05,3835.616663
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
25%,1.980000e+02,7.000000e+00,8.000000e+01,3.000000
50%,3.310000e+02,1.700000e+01,2.940000e+02,8.000000
75%,5.860000e+02,3.300000e+01,7.110000e+02,27.000000
max,3.011122e+07,1.722763e+06,2.767511e+07,236489.000000


In [33]:
recipes.loc[recipes.calories > 10000]

,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title
1308,4157357.0,"[Lamb, Rice, Vegetable, Braise, Dinner, Raisin...",2007-08-24 18:04:29.000,"This lamb-rich, Uzbek-style pilaf is typical o...",[Wash rice in several changes of cold water in...,221495.0,"[3 cups basmati rice (7 ounces), 1 medium onio...",236489.0,5.000,3134853.0,"Rice Pilaf with Lamb, Carrots, and Raisins"
1843,12010.0,"[Milk/Cream, Beef, Fish, Onion, Bake, Vinegar,...",2004-08-20 04:00:00.000,None,"[In a heavy saucepan boil vinegar, water, onio...",1115.0,"[3/4 cup balsamic vinegar, 1 1/2 cups water, 2...",403.0,4.375,18212.0,Bacon-Wrapped Trout Stuffed with Balsamic Onio...
2985,4518216.0,"[Food Processor, Backyard BBQ, Ground Lamb, Sp...",2008-05-12 21:04:07.000,Köfte are a street-food favorite in the Middle...,"[Soak bread in water to cover 30 minutes, then...",44198.0,"[3 slices firm white sandwich bread, torn into...",166471.0,5.000,7540990.0,Lamb Köfte with Tarator Sauce
3290,19576.0,"[Beef, Braise, Beef Rib, Red Wine, Fall, Bon A...",2004-08-20 04:00:00.000,Tiring of the ultra-refined nouvelle cuisine t...,[Stir flour and allspice in medium bowl to ble...,1818.0,"[1/4 cup all-purpose flour, 1 teaspoon (genero...",723.0,3.750,2950.0,Braised Short Ribs with Red Wine Gravy
5165,3358029.0,"[Fruit, Dessert, Bake, Christmas, Thanksgiving...",2004-08-20 04:00:00.000,None,[Position rack in center of oven and preheat t...,186642.0,"[1/2 cup applejack brandy or brandy, 1/2 cup d...",58324.0,3.125,3449373.0,Caramelized Apple and Pear Pie
6400,29997918.0,"[Egg, Fruit, Bake, Blueberry, Summer, Tapioca,...",2006-07-20 20:48:24.000,If you've never had pie made from wild blueber...,[Put a large baking sheet on oven rack in midd...,1716279.0,"[1 1/4 cups packed light brown sugar, 5 tables...",200210.0,4.375,27570999.0,Deep-Dish Wild Blueberry Pie
6678,16050.0,"[Beef, Citrus, Fruit, Ginger, Braise, Super Bo...",2004-08-20 04:00:00.000,Hoisin sauce is sold at Asian markets and in t...,[Preheat oven to 300°F. Using vegetable peeler...,1480.0,"[4 large oranges, 3/4 cup hoisin sauce, 1/3 cu...",592.0,4.375,2629.0,Braised Orange-Ginger Short Ribs with Dried Ap...
6945,3358273.0,"[Chocolate, Nut, Dessert, Bake, Christmas, Tha...",2004-08-20 04:00:00.000,None,[Position rack in center of oven and preheat t...,186660.0,[4 ounces bittersweet (not unsweetened) or sem...,58334.0,3.125,3449512.0,Chocolate-Almond Pie
8232,24117.0,"[Lamb, Mushroom, Mustard, Quick & Easy, Father...",2009-04-14 04:00:00.000,"Rubbed all over with a mixture of garlic, fres...",[Prepare grill for direct-heat cooking over me...,2228.0,"[2 ounces dried porcini mushrooms, 2 cups boil...",940.0,5.000,4382.0,Grilled Lamb Chops with Porcini Mustard
8917,22312.0,"[Chicken, Quick & Easy, Backyard BBQ, Spice, G...",2004-08-20 04:00:00.000,Can be prepared in 45 minutes or less.,"[Make salsa., Prepare grill., Pat chicken dry ...",1007.0,[3 whole boneless chicken breasts with skin (a...,332.0,3.750,13820.0,Grilled Cumin Chicken Breasts with Avocado Salsa


In [122]:
list(no_nutri.title)[:20]

['Mahi-Mahi in Tomato Olive Sauce ',
 'Ham and Spring Vegetable Salad with Shallot Vinaigrette ',
 'Spicy-Sweet Kumquats ',
 'Spicy Noodle Soup ',
 '"Fried" Chicken ',
 'Fish Fillets in Parchment with Asparagus and Orange ',
 'Pancetta and Taleggio Lasagna with Treviso ',
 'Asian Steak Topped with Bell Pepper Stir-Fry ',
 'Moroccan-Style Preserved Lemons ',
 'Fontina Mac with Squash and Sage ',
 'Pastry Twists with Spiced Sugar-Honey Glaze ',
 'Honey Rye ',
 'Purple-Potato and Crab Gratin ',
 'Grilled Beef, Jícama, and Apple Salad ',
 'Herbed Goat Cheese Spread with Mint ',
 'Braised Chicken With Artichokes and Olives ',
 'Collard-and-Prosciutto Chicken Roulades Over Watercress Salad ',
 'Orange-Almond Cake with Chocolate Icing ',
 'Anadama Rolls with Mixed Seeds ',
 'Bacon Crackers ']

In [33]:
recipes.title[0].strip().replace(' ','%20')

'Lentil,%20Apple,%20and%20Turkey%20Wrap'

In [137]:
import unidecode

In [190]:
list(no_nutri.title)[:20]

['Mahi-Mahi in Tomato Olive Sauce ',
 'Ham and Spring Vegetable Salad with Shallot Vinaigrette ',
 'Spicy-Sweet Kumquats ',
 'Spicy Noodle Soup ',
 '"Fried" Chicken ',
 'Fish Fillets in Parchment with Asparagus and Orange ',
 'Pancetta and Taleggio Lasagna with Treviso ',
 'Asian Steak Topped with Bell Pepper Stir-Fry ',
 'Moroccan-Style Preserved Lemons ',
 'Fontina Mac with Squash and Sage ',
 'Pastry Twists with Spiced Sugar-Honey Glaze ',
 'Honey Rye ',
 'Purple-Potato and Crab Gratin ',
 'Grilled Beef, Jícama, and Apple Salad ',
 'Herbed Goat Cheese Spread with Mint ',
 'Braised Chicken With Artichokes and Olives ',
 'Collard-and-Prosciutto Chicken Roulades Over Watercress Salad ',
 'Orange-Almond Cake with Chocolate Icing ',
 'Anadama Rolls with Mixed Seeds ',
 'Bacon Crackers ']

In [76]:
no_nutri[no_nutri.duplicated('title')]
no_nutri.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4135 entries, 3 to 20120
Data columns (total 11 columns):
calories       0 non-null float64
categories     4135 non-null object
date           4135 non-null datetime64[ns]
desc           2852 non-null object
directions     4135 non-null object
fat            0 non-null float64
ingredients    4135 non-null object
protein        0 non-null float64
rating         4131 non-null float64
sodium         0 non-null float64
title          4135 non-null object
dtypes: datetime64[ns](1), float64(5), object(5)
memory usage: 387.7+ KB


In [80]:
# test.sort_values('rating', ascending=False).drop_duplicates('title').sort_index().rating.describe()
no_nutri.sort_values('rating', ascending = False, inplace = True)

C:\Users\vuong\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [96]:
r = requests.get('https://www.epicurious.com/search/Tomato%20Jam?content=recipe')
s = bs(r.text, 'lxml')

True


In [231]:
# print(base_url + s.find_all('a', string='Tomato Jam')['href'])
for h in s.find_all('a', string = 'Tomato Jam'): print(h['href'])

/recipes/food/views/tomato-jam-366752
/recipes/food/views/tomato-jam-358809
/recipes/food/views/tomato-jam-242972


In [196]:
recs = [e.strip().replace('"', '') for e in list(no_nutri.title)]


In [73]:
no_nutri[no_nutri['title'] == 'Homemade Chili Powder ']

,calories,categories,date,desc,directions,fat,ingredients,protein,rating,sodium,title
1264,NaN,"[Garlic, Spice, Oregano, Cumin, Chile Pepper]",2009-10-15 04:00:00,When you combine powdered chiles with cumin an...,[Combine all of the ingredients in a small mix...,NaN,"[1 cup ancho powder, 1 teaspoon dried Mexican ...",NaN,0.0,NaN,Homemade Chili Powder


In [194]:
recs

['Mahi-Mahi in Tomato Olive Sauce',
 'Ham and Spring Vegetable Salad with Shallot Vinaigrette',
 'Spicy-Sweet Kumquats',
 'Spicy Noodle Soup',
 'Fried Chicken',
 'Fish Fillets in Parchment with Asparagus and Orange',
 'Pancetta and Taleggio Lasagna with Treviso',
 'Asian Steak Topped with Bell Pepper Stir-Fry',
 'Moroccan-Style Preserved Lemons',
 'Fontina Mac with Squash and Sage',
 'Pastry Twists with Spiced Sugar-Honey Glaze',
 'Honey Rye',
 'Purple-Potato and Crab Gratin',
 'Grilled Beef, Jícama, and Apple Salad',
 'Herbed Goat Cheese Spread with Mint',
 'Braised Chicken With Artichokes and Olives',
 'Collard-and-Prosciutto Chicken Roulades Over Watercress Salad',
 'Orange-Almond Cake with Chocolate Icing',
 'Anadama Rolls with Mixed Seeds',
 'Bacon Crackers',
 'Ricotta Omelets',
 'Carrot, Snow Pea, and Red Pepper Julienne in Honey Vinaigrette',
 'Grilled Steak Salad with Beets and Scallions',
 'Short Rib Pot Pie',
 'Sausage Fennel Stuffing',
 'Baked Pears with Rosemary, Gorgonzola

In [214]:
base_url = 'https://www.epicurious.com'
rec_urls = []

In [203]:
t = f"{base_url}search/{recs[13].replace(' ', '%20')}?content=recipe&page=1"
print(t)
d = f"^/recipes/food/views/{unidecode.unidecode(recs[13].replace(' ', '-').lower())}-[0-9]{{1,6}}"
print(d)
re.compile(d)

https://www.epicurious.com/search/Grilled%20Beef,%20Jícama,%20and%20Apple%20Salad?content=recipe&page=1
^/recipes/food/views/grilled-beef,-jicama,-and-apple-salad-[0-9]{1,6}


re.compile(r'^/recipes/food/views/grilled-beef,-jicama,-and-apple-salad-[0-9]{1,6}',
re.UNICODE)

In [197]:
def get_url(terms):
    
#     search_urls = [base_url + term.replace(' ', '%20') for term in terms]
    for term in terms:
        page = 1
        while True:
            search_url = f"{base_url}search/{term.replace(' ', '%20')}?content=recipe&page={page}"
            regex = f"^/recipes/food/views/{unidecode.unidecode(term.replace(' ', '-').lower())}-[0-9]{{1,6}}"
#         print(search_url, regex)
            req = requests.get(search_url)
            soup = bs(req.text, 'lxml')
        
            try:
                rec_urls.append(base_url + soup.find('a', attrs={'href': re.compile(regex)})['href'])
                break
            except TypeError:
                page += 1
            if page == 5:
                print('Unsuccessful')
                break
        print(f"{rec_urls[-1]} : {len(rec_urls) - 1}")
            

In [198]:
# get_url(list(no_nutri.title)[0])
# get_url([no_nutri.title.iloc[0]])
get_url(recs)

https://www.epicurious.com//recipes/food/views/mahi-mahi-in-tomato-olive-sauce-352551 : 0
https://www.epicurious.com//recipes/food/views/ham-and-spring-vegetable-salad-with-shallot-vinaigrette-5212 : 1
https://www.epicurious.com//recipes/food/views/spicy-sweet-kumquats-102726 : 2
https://www.epicurious.com//recipes/food/views/spicy-noodle-soup-103188 : 3
Unsuccessful
https://www.epicurious.com//recipes/food/views/spicy-noodle-soup-103188 : 3
https://www.epicurious.com//recipes/food/views/fish-fillets-in-parchment-with-asparagus-and-orange-241849 : 4
https://www.epicurious.com//recipes/food/views/pancetta-and-taleggio-lasagna-with-treviso-356354 : 5
https://www.epicurious.com//recipes/food/views/asian-steak-topped-with-bell-pepper-stir-fry-350398 : 6
https://www.epicurious.com//recipes/food/views/moroccan-style-preserved-lemons-242341 : 7
https://www.epicurious.com//recipes/food/views/fontina-mac-with-squash-and-sage-51206440 : 8
https://www.epicurious.com//recipes/food/views/pastry-twi

KeyboardInterrupt: 

In [195]:
rec_urls

[]

In [44]:
re.compile('^/recipes/food/views/Mahi-Mahi-in-Tomato-Olive-Sauce-[0-9]{6}')

re.compile(r'^/recipes/food/views/Mahi-Mahi-in-Tomato-Olive-Sauce-[0-9]{6}',
re.UNICODE)

In [35]:
no_nutri.title.iloc[0]

'Mahi-Mahi in Tomato Olive Sauce '

In [17]:
help(str.replace)

Help on method_descriptor:

replace(...)
    S.replace(old, new[, count]) -> str
    
    Return a copy of S with all occurrences of substring
    old replaced by new.  If the optional argument count is
    given, only the first count occurrences are replaced.

